In [1]:
import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from torch.utils.data import DataLoader
# from dataset.dataloader import CINE2DT
# from model.model_pytorch import CRNN_MRI
# from utils import multicoil2single, compressed_sensing as cs,IFFT2c
# from ..utils import multicoil2single, compressed_sensing as cs
# from utils.dnn_io import to_tensor_format, from_tensor_format
# from utils.fastmriBaseUtils import FFT2c,IFFT2c
# from trainer_dcrnn_test import prep_input
from torch.autograd import Variable
import os 
from PIL import Image
# import scipy.io as sio
from scipy.io import loadmat
import scipy.io as scio
import h5py

os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # 指定使用 GPU 1 和 GPU 4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [6]:
import medutils
print(dir(medutils))  # 查看所有可用子模块

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [14]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from matplotlib import gridspec

def center_crop(image, target_shape):
    """自定义中心裁剪函数"""
    if image.ndim == 2:
        h, w = image.shape
        target_h, target_w = target_shape
        start_h = (h - target_h) // 2
        start_w = (w - target_w) // 2
        return image[start_h:start_h+target_h, start_w:start_w+target_w]
    elif image.ndim == 3:
        t, h, w = image.shape
        target_h, target_w = target_shape
        start_h = (h - target_h) // 2
        start_w = (w - target_w) // 2
        return image[:, start_h:start_h+target_h, start_w:start_w+target_w]
    else:
        raise ValueError("Unsupported image dimensions")

def load_4d_data(file_path, config):
    """处理四维数据 (slices=118, time=18, H=192, W=192)"""
    data = np.load(file_path)
    
    # 验证数据维度
    if data.shape != (118, 18, 192, 192):
        raise ValueError(f"数据维度错误: 应为 (118,18,192,192)，实际得到 {data.shape}")
    
    # 提取指定切片和时间点
    slice_idx = np.clip(config['slice_index'], 0, 117)
    time_idx = np.clip(config['time_index'], 0, 17)
    
    # 提取并归一化图像
    image = np.abs(data[slice_idx, time_idx])
    return (image - image.min()) / (image.max() - image.min())

# import yaml
# import numpy as np
# import matplotlib.pyplot as plt
# from medutils.visualization import center_crop
# from scipy.interpolate import interp1d
# from matplotlib import gridspec

# def load_4d_data(file_path, config):
#     """处理正确维度的4D数据 (slices=118, time=18, H=192, W=192)"""
#     data = np.load(file_path)
    
#     # 验证数据维度
#     if data.shape != (118, 18, 192, 192):
#         raise ValueError(f"Invalid data shape: {data.shape}, expected (118, 18, 192, 192)")
    
#     # 提取指定切片和时间点
#     slice_idx = np.clip(config['slice_index'], 0, 117)
#     time_idx = np.clip(config['time_index'], 0, 17)
    
#     # 提取二维图像并归一化
#     image = np.abs(data[slice_idx, time_idx])
#     return (image - image.min()) / (image.max() - image.min())
def load_config(config_file):
    try:
        with open(config_file, 'r') as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        raise Exception(f"配置文件 {config_file} 未找到")
    except yaml.YAMLError as exc:
        raise Exception(f"配置文件解析错误: {str(exc)}")

In [ ]:

# def visualize_results(config_path):
#     # 加载配置
#     config = load_config(config_path)['my_config']
#     # config = load_config(config_path)
    
#     # 加载参考数据
#     ref_image = load_4d_data(config['reference_file'], config) if config.get('reference_file') else None
    
#     # 加载对比数据
#     comp_images = []
#     for fpath in config['comparison_files']:
#         try:
#             img = load_4d_data(fpath, config)
#             comp_images.append(img)
#         except Exception as e:
#             print(f"Error loading {fpath}: {str(e)}")
#             comp_images.append(np.zeros((192,192)))  # 用空白图占位
    
#     # 统一图像尺寸
#     base_shape = (192, 192)
#     if ref_image is not None:
#         ref_image = center_crop(ref_image, base_shape)
#     comp_images = [center_crop(img, base_shape) for img in comp_images]
    
#     # 创建可视化布局
#     # num_cols = len(comp_images) + (1 if ref_image else 0)
#     # 修正行：将判断条件改为检查 None
#     num_cols = len(comp_images) + (1 if ref_image is not None else 0)
#     fig = plt.figure(figsize=(num_cols*3, 6))  # 动态调整宽度
    
#     # 主图布局
#     gs_main = gridspec.GridSpec(2, num_cols, height_ratios=[3,1])
    
#     # 绘制主图
#     def plot_slice(ax, img, title):
#         im = ax.imshow(img, cmap='gray', 
#                       vmin=config['display_range'][0], 
#                       vmax=config['display_range'][1])
#         ax.set_title(title, fontsize=10)
#         ax.axis('off')
#         return im
    
#     # 绘制参考图
#     if ref_image is not None:
#         ax_ref = plt.subplot(gs_main[0, 0])
#         im_ref = plot_slice(ax_ref, ref_image, "Reference")
    
#     # 绘制对比图
#     for idx, (img, title) in enumerate(zip(comp_images, config['comparison_titles'])):
#         # col = idx + (1 if ref_image else 0)
#         col = idx + (1 if ref_image is not None else 0)
#         ax = plt.subplot(gs_main[0, col])
#         plot_slice(ax, img, title)
    
#     # 绘制ROI区域
#     roi = config['roi_coordinates']
#     ax_roi = plt.subplot(gs_main[1, :])
#     ax_roi.imshow(ref_image[roi[1]:roi[3], roi[0]:roi[2]], cmap='gray')
#     ax_roi.set_title("ROI Zoom", fontsize=8)
#     ax_roi.axis('off')
    
#     # 添加颜色条
#     cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
#     plt.colorbar(im_ref, cax=cax)
    
#     # 保存结果
#     plt.savefig(f'result_s{config["slice_index"]}_t{config["time_index"]}.png', 
#                 dpi=300, bbox_inches='tight')
#     plt.close()

def visualize_results(config_path):
    # 加载配置
    config = load_config(config_path)['my_config']
    
    # 加载参考数据
    ref_image = load_4d_data(config['reference_file'], config) if config.get('reference_file') else None
    
    # 加载对比数据
    comp_images = []
    for fpath in config['comparison_files']:
        try:
            img = load_4d_data(fpath, config)
            comp_images.append(img)
        except Exception as e:
            print(f"Error loading {fpath}: {str(e)}")
            comp_images.append(np.zeros((192,192)))

    # 统一图像尺寸
    base_shape = (192, 192)
    if ref_image is not None:
        ref_image = center_crop(ref_image, base_shape)
    comp_images = [center_crop(img, base_shape) for img in comp_images]
    
    # 计算差异图（需要参考图像存在）
    error_images = []
    if ref_image is not None:
        for comp_img in comp_images:
            error = np.abs(ref_image - comp_img)
            error_images.append((error - error.min()) / (error.max() - error.min()))
    
    # 创建三行布局
    num_cols = len(comp_images)
    fig = plt.figure(figsize=(num_cols*3, 9))  # 宽度随方法数量变化，固定高度
    
    # 三行布局：原始图像、ROI区域、差异图
    gs = gridspec.GridSpec(3, num_cols, height_ratios=[3, 1, 1])
    roi = config['roi_coordinates']
    
    # 第一行：原始图像
    for col in range(num_cols):
        ax = plt.subplot(gs[0, col])
        img = comp_images[col]
        im = ax.imshow(img, cmap='gray', 
                      vmin=config['display_range'][0], 
                      vmax=config['display_range'][1])
        ax.set_title(config['comparison_titles'][col], fontsize=12)
        ax.axis('off')
    
    # 第二行：各方法ROI区域
    for col in range(num_cols):
        ax = plt.subplot(gs[1, col])
        img = comp_images[col]
        roi_img = img[roi[1]:roi[3], roi[0]:roi[2]]
        ax.imshow(roi_img, cmap='gray', 
                 vmin=config['display_range'][0],
                 vmax=config['display_range'][1])
        ax.axis('off')
        if col == 0:
            ax.set_ylabel("ROI", fontsize=10, rotation=0, ha='right', va='center')
    
    # 第三行：差异图（需要参考图像存在）
    if ref_image is not None:
        for col in range(num_cols):
            ax = plt.subplot(gs[2, col])
            error_img = error_images[col]
            im_error = ax.imshow(error_img, cmap='viridis',
                                vmin=config['error_display_range'][0],
                                vmax=config['error_display_range'][1])
            ax.axis('off')
            if col == 0:
                ax.set_ylabel("Error", fontsize=10, rotation=0, ha='right', va='center')
    
    # 添加颜色条
    plt.subplots_adjust(right=0.85)
    
    # 主图颜色条
    cax1 = fig.add_axes([0.88, 0.65, 0.015, 0.25])
    plt.colorbar(im, cax=cax1, label='Intensity')
    
    # 差异图颜色条
    if ref_image is not None:
        cax2 = fig.add_axes([0.88, 0.15, 0.015, 0.25])
        plt.colorbar(im_error, cax=cax2, label='Error')
    
    # 保存结果
    plt.savefig(f'comparison_result_s{config["slice_index"]}_t{config["time_index"]}_3.png', 
                dpi=300, bbox_inches='tight')
    plt.close()

In [42]:
# # # def visualize_results(config_path):
# # #     # 加载配置
# # #     config = load_config(config_path)['my_config']
    
# # #     # 加载参考数据
# # #     ref_image = load_4d_data(config['reference_file'], config) if config.get('reference_file') else None
    
# # #     # 加载对比数据
# # #     comp_images = []
# # #     for fpath in config['comparison_files']:
# # #         try:
# # #             img = load_4d_data(fpath, config)
# # #             comp_images.append(img)
# # #         except Exception as e:
# # #             print(f"Error loading {fpath}: {str(e)}")
# # #             comp_images.append(np.zeros((192,192)))

# # #     # 统一图像尺寸
# # #     base_shape = (192, 192)
# # #     if ref_image is not None:
# # #         ref_image = center_crop(ref_image, base_shape)
# # #     comp_images = [center_crop(img, base_shape) for img in comp_images]
    
# # #     # 计算差异图（需要参考图像存在）
# # #     error_images = []
# # #     if ref_image is not None:
# # #         error_images.append(np.zeros_like(ref_image))  # 为参考图像预留空白误差图位置
# # #         for comp_img in comp_images:
# # #             error = np.abs(ref_image - comp_img)
# # #             error_images.append((error - error.min()) / (error.max() - error.min()))
    
# # #     # 创建四列布局（参考图像 + 三种方法）
# # #     num_methods = len(comp_images)
# # #     num_cols = num_methods + 1  # 增加参考图像列
# # #     fig = plt.figure(figsize=(num_cols*3, 9))
    
# # #     # 三行布局：原始图像、ROI区域、差异图
# # #     gs = gridspec.GridSpec(3, num_cols, height_ratios=[3, 1, 1])
# # #     roi = config['roi_coordinates']
    
# # #     # 第一行：参考图像和方法图像
# # #     def plot_main_image(ax, img, title, is_ref=False):
# # #         im = ax.imshow(img, cmap='gray', 
# # #                       vmin=config['display_range'][0], 
# # #                       vmax=config['display_range'][1])
# # #         ax.set_title(title, fontsize=12)
# # #         ax.axis('off')
# # #         return im
    
# # #     # 绘制参考图像
# # #     if ref_image is not None:
# # #         ax_ref = plt.subplot(gs[0, 0])
# # #         im_ref = plot_main_image(ax_ref, ref_image, "Reference", is_ref=True)
    
# # #     # 绘制方法图像
# # #     for col in range(1, num_cols):
# # #         ax = plt.subplot(gs[0, col])
# # #         img = comp_images[col-1]
# # #         im = plot_main_image(ax, img, config['comparison_titles'][col-1])
    
# # #     # 第二行：ROI区域
# # #     def plot_roi(ax, img, is_ref=False):
# # #         roi_img = img[roi[1]:roi[3], roi[0]:roi[2]]
# # #         ax.imshow(roi_img, cmap='gray',
# # #                  vmin=config['display_range'][0],
# # #                  vmax=config['display_range'][1])
# # #         ax.axis('off')
# # #         if is_ref:
# # #             ax.set_ylabel("ROI", fontsize=10, rotation=0, ha='right', va='center')
    
# # #     # 参考图像ROI
# # #     if ref_image is not None:
# # #         ax_ref_roi = plt.subplot(gs[1, 0])
# # #         plot_roi(ax_ref_roi, ref_image, is_ref=True)
    
# # #     # 方法ROI
# # #     for col in range(1, num_cols):
# # #         ax = plt.subplot(gs[1, col])
# # #         plot_roi(ax, comp_images[col-1])
    
# # #     # 第三行：差异图
# # #     def plot_error(ax, error_img, title):
# # #         im = ax.imshow(error_img, cmap='viridis',
# # #                       vmin=config['error_display_range'][0],
# # #                       vmax=config['error_display_range'][1])
# # #         ax.axis('off')
# # #         return im
    
# # #     if ref_image is not None:
# # #         # 参考图像位置留空
# # #         ax_ref_error = plt.subplot(gs[2, 0])
# # #         ax_ref_error.axis('off')
        
# # #         # 方法差异图
# # #         for col in range(1, num_cols):
# # #             ax = plt.subplot(gs[2, col])
# # #             im_error = plot_error(ax, error_images[col], config['comparison_titles'][col-1])
    
# # #     # 添加颜色条
# # #     plt.subplots_adjust(right=0.85)
    
# # #     # 主图颜色条
# # #     cax1 = fig.add_axes([0.88, 0.65, 0.015, 0.25])
# # #     plt.colorbar(im_ref, cax=cax1, label='Intensity')
    
# # #     # 差异图颜色条
# # #     if ref_image is not None:
# # #         cax2 = fig.add_axes([0.88, 0.15, 0.015, 0.25])
# # #         plt.colorbar(im_error, cax=cax2, label='Error')
    
# # #     # 保存结果
# # #     plt.savefig(f'comparison_result_s{config["slice_index"]}_t{config["time_index"]}_vista_1.png', 
# # #                 dpi=300, bbox_inches='tight')
# # #     plt.close()

# # def visualize_results(config_path):
# #     # ... [保持配置加载和数据预处理部分不变] ...
# #     # 加载配置
# #     config = load_config(config_path)['my_config']
    
# #     # 加载参考数据
# #     ref_image = load_4d_data(config['reference_file'], config) if config.get('reference_file') else None
    
# #     # 加载对比数据
# #     comp_images = []
# #     for fpath in config['comparison_files']:
# #         try:
# #             img = load_4d_data(fpath, config)
# #             comp_images.append(img)
# #         except Exception as e:
# #             print(f"Error loading {fpath}: {str(e)}")
# #             comp_images.append(np.zeros((192,192)))

# #     # 统一图像尺寸
# #     base_shape = (192, 192)
# #     if ref_image is not None:
# #         ref_image = center_crop(ref_image, base_shape)
# #     comp_images = [center_crop(img, base_shape) for img in comp_images]
    
# #     # 计算差异图（需要参考图像存在）
# #     error_images = []
# #     if ref_image is not None:
# #         error_images.append(np.zeros_like(ref_image))  # 为参考图像预留空白误差图位置
# #         for comp_img in comp_images:
# #             error = np.abs(ref_image - comp_img)
# #             error_images.append((error - error.min()) / (error.max() - error.min()))

# #     # 新增旋转函数
# #     def rotate_image(img):
# #         """顺时针旋转90度"""
# #         return np.rot90(img, k=1)

# #     # 创建四列布局（参考图像 + 三种方法）
# #     num_methods = len(comp_images)
# #     num_cols = num_methods + 1  # 参考图像占第一列
# #     fig = plt.figure(figsize=(9, num_cols*3))  # 调整为纵向布局
    
# #     # 三行布局：原始图像、ROI区域、差异图（保持行列方向不变）
# #     gs = gridspec.GridSpec(3, num_cols, height_ratios=[3, 1, 1])
    
# #     # 第一行：旋转后的参考图像和方法图像
# #     # 绘制参考图像
# #     ax_ref = plt.subplot(gs[0, 0])
# #     rotated_ref = rotate_image(ref_image)
# #     im_ref = ax_ref.imshow(rotated_ref, cmap='gray',
# #                           vmin=config['display_range'][0],
# #                           vmax=config['display_range'][1])
# #     ax_ref.set_title("Reference", fontsize=12)
    
# #     # 绘制方法图像
# #     for col in range(1, num_cols):
# #         ax = plt.subplot(gs[0, col])
# #         rotated_img = rotate_image(comp_images[col-1])
# #         ax.imshow(rotated_img, cmap='gray',
# #                  vmin=config['display_range'][0],
# #                  vmax=config['display_range'][1])
# #         ax.set_title(config['comparison_titles'][col-1], fontsize=12)
# #         ax.axis('off')

# #     # 第二行：旋转后的ROI区域
# #     # 参考ROI
# #     ax_ref_roi = plt.subplot(gs[1, 0])
# #     rotated_ref_roi = rotate_image(ref_image[roi[1]:roi[3], roi[0]:roi[2]])
# #     ax_ref_roi.imshow(rotated_ref_roi, cmap='gray',
# #                      vmin=config['display_range'][0],
# #                      vmax=config['display_range'][1])
    
# #     # 方法ROI
# #     for col in range(1, num_cols):
# #         ax = plt.subplot(gs[1, col])
# #         rotated_roi = rotate_image(comp_images[col-1][roi[1]:roi[3], roi[0]:roi[2]])
# #         ax.imshow(rotated_roi, cmap='gray',
# #                  vmin=config['display_range'][0],
# #                  vmax=config['display_range'][1])
# #         ax.axis('off')

# #     # 第三行：旋转后的差异图
# #     if ref_image is not None:
# #         # 参考位置留空
# #         plt.subplot(gs[2, 0]).axis('off')
        
# #         # 方法差异图
# #         for col in range(1, num_cols):
# #             ax = plt.subplot(gs[2, col])
# #             rotated_error = rotate_image(error_images[col-1])
# #             im_error = ax.imshow(rotated_error, cmap='viridis',
# #                                 vmin=config['error_display_range'][0],
# #                                 vmax=config['error_display_range'][1])
# #             ax.axis('off')

# #     # 调整颜色条位置为横向
# #     cax1 = fig.add_axes([0.15, 0.92, 0.7, 0.02])  # 主图颜色条在上方
# #     plt.colorbar(im_ref, cax=cax1, orientation='horizontal', label='Intensity')
    
# #     cax2 = fig.add_axes([0.15, 0.01, 0.7, 0.02])  # 误差图颜色条在下方
# #     plt.colorbar(im_error, cax=cax2, orientation='horizontal', label='Error')

# #     plt.savefig('rotated_comparison.png', dpi=300, bbox_inches='tight')
# #     plt.close()

# import yaml
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import gridspec

# def center_crop(image, target_shape):
#     """自定义中心裁剪函数"""
#     if image.ndim == 2:
#         h, w = image.shape
#         target_h, target_w = target_shape
#         start_h = (h - target_h) // 2
#         start_w = (w - target_w) // 2
#         return image[start_h:start_h+target_h, start_w:start_w+target_w]
#     else:
#         raise ValueError("Unsupported image dimensions")

# def load_4d_data(file_path, config):
#     """处理四维数据 (slices=118, time=18, H=192, W=192)"""
#     data = np.load(file_path)
    
#     # 验证数据维度
#     if data.shape != (118, 18, 192, 192):
#         raise ValueError(f"数据维度错误: 应为 (118,18,192,192)，实际得到 {data.shape}")
    
#     # 提取指定切片和时间点
#     slice_idx = np.clip(config['slice_index'], 0, 117)
#     time_idx = np.clip(config['time_index'], 0, 17)
    
#     # 提取并归一化图像
#     image = np.abs(data[slice_idx, time_idx])
#     img_norm = (image - image.min()) / (image.max() - image.min())
#     brightness_factor = 3
#     img_brightened = np.clip(img_norm * brightness_factor, 0, 1)
#     return img_brightened

# def load_config(config_file):
#     try:
#         with open(config_file, 'r') as file:
#             return yaml.safe_load(file)
#     except Exception as e:
#         raise Exception(f"配置文件错误: {str(e)}")

# def rotate_image(img):
#     """顺时针旋转90度"""
#     # return np.rot90(img, k=1)
#     return np.rot90(img, k=-1)

# def visualize_results(config_path,last_name):
#     # 加载配置
#     config = load_config(config_path)['my_config']
    
#     # 加载参考数据
#     ref_image = load_4d_data(config['reference_file'], config) if config.get('reference_file') else None
    
#     # 加载对比数据
#     comp_images = []
#     for fpath in config['comparison_files']:
#         try:
#             img = load_4d_data(fpath, config)
#             comp_images.append(img)
#         except Exception as e:
#             print(f"Error loading {fpath}: {str(e)}")
#             comp_images.append(np.zeros((192,192)))

#     # 统一图像尺寸
#     base_shape = (192, 192)
#     if ref_image is not None:
#         ref_image = center_crop(ref_image, base_shape)
#     comp_images = [center_crop(img, base_shape) for img in comp_images]
    
#     # 计算差异图（需要参考图像存在）
#     error_images = []
#     if ref_image is not None:
#         for comp_img in comp_images:
#             error = np.abs(ref_image - comp_img)
#             error_images.append((error - error.min()) / (error.max() - error.min()))
    
#     # 创建布局
#     num_methods = len(comp_images)
#     num_cols = num_methods + 1  # 参考图像占第一列
#     fig = plt.figure(figsize=(num_cols*3, 9))
    
#     # 三行布局：原始图像、ROI、差异图
#     gs = gridspec.GridSpec(3, num_cols, height_ratios=[3, 1, 1], hspace=0.15, wspace=0.05)
#     roi = config['roi_coordinates']
    
#     # ========== 第一行：原始图像 ==========
#     # 参考图像
#     ax_ref = plt.subplot(gs[0, 0])
#     rotated_ref = rotate_image(ref_image)
#     im_ref = ax_ref.imshow(rotated_ref, cmap='gray', 
#                           vmin=config['display_range'][0], 
#                           vmax=config['display_range'][1])
#     ax_ref.set_title("Reference", fontsize=12, pad=10)
#     ax_ref.axis('off')
    
#     # 对比方法图像
#     for col in range(1, num_cols):
#         ax = plt.subplot(gs[0, col])
#         rotated_img = rotate_image(comp_images[col-1])
#         ax.imshow(rotated_img, cmap='gray',
#                  vmin=config['display_range'][0],
#                  vmax=config['display_range'][1])
#         ax.set_title(config['comparison_titles'][col-1], fontsize=12, pad=10)
#         ax.axis('off')
    
#     # ========== 第二行：ROI区域 ==========
#     # 参考ROI
#     ax_ref_roi = plt.subplot(gs[1, 0])
#     ref_roi = ref_image[roi[1]:roi[3], roi[0]:roi[2]]
#     rotated_ref_roi = rotate_image(ref_roi)
#     ax_ref_roi.imshow(rotated_ref_roi, cmap='gray',
#                      vmin=config['display_range'][0],
#                      vmax=config['display_range'][1])
#     ax_ref_roi.set_ylabel("ROI", fontsize=10, rotation=0, labelpad=20, va='center', ha='right')
#     ax_ref_roi.axis('off')
    
#     # 方法ROI
#     for col in range(1, num_cols):
#         ax = plt.subplot(gs[1, col])
#         comp_roi = comp_images[col-1][roi[1]:roi[3], roi[0]:roi[2]]
#         rotated_roi = rotate_image(comp_roi)
#         ax.imshow(rotated_roi, cmap='gray',
#                  vmin=config['display_range'][0],
#                  vmax=config['display_range'][1])
#         ax.axis('off')
    
#     # ========== 第三行：差异图 ==========
#     if ref_image is not None:
#         # 参考位置留空
#         plt.subplot(gs[2, 0]).axis('off')
        
#         # 方法差异图
#         for col in range(1, num_cols):
#             ax = plt.subplot(gs[2, col])
#             rotated_error = rotate_image(error_images[col-1])
#             im_error = ax.imshow(rotated_error, cmap='viridis',
#                                 vmin=config['error_display_range'][0],
#                                 vmax=config['error_display_range'][1])
#             ax.axis('off')
#             if col == 1:
#                 ax.set_ylabel("Error", fontsize=10, rotation=0, labelpad=20, va='center', ha='right')
    
#     # ========== 颜色条 ==========
#     plt.subplots_adjust(right=0.85)
#     cax1 = fig.add_axes([0.88, 0.65, 0.015, 0.25])
#     plt.colorbar(im_ref, cax=cax1, label='Intensity')
    
#     if ref_image is not None:
#         cax2 = fig.add_axes([0.88, 0.15, 0.015, 0.25])
#         plt.colorbar(im_error, cax=cax2, label='Error')
    
#     # 保存结果
#     # plt.savefig(
#     #     # f'result_rotated_s{config["slice_index"]}_t{config["time_index"]}.png',
#     #     f'result_rotated_s{config["slice_index"]}_t{config["time_index"]}_{last_name}.png',  # 添加_{lastname}
#     #     dpi=300,
#     #     bbox_inches='tight'
#     # )
#     output_dir = "result_img"
#     os.makedirs(output_dir, exist_ok=True)  # 自动创建目录

#     plt.savefig(
#         os.path.join(output_dir, f'result_rotated_s{config["slice_index"]}_t{config["time_index"]}_{last_name}.png'),
#         dpi=300,
#         bbox_inches='tight'
#     )
#     plt.close()

import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

def center_crop(image, target_shape):
    """自定义中心裁剪函数（支持2D图像）"""
    h, w = image.shape
    target_h, target_w = target_shape
    start_h = (h - target_h) // 2
    start_w = (w - target_w) // 2
    return image[start_h:start_h+target_h, start_w:start_w+target_w]

def load_4d_data(file_path, config):
    """加载并预处理四维MRI数据 (slices=118, time=18, H=192, W=192)"""
    data = np.load(file_path)
    
    # 验证数据维度
    if data.shape != (118, 18, 192, 192):
        raise ValueError(f"数据维度错误: 应为 (118,18,192,192)，实际得到 {data.shape}")
    
    # 提取指定切片和时间点
    slice_idx = np.clip(config['slice_index'], 0, 117)
    time_idx = np.clip(config['time_index'], 0, 17)
    
    # 提取并增强图像
    # image = np.abs(data[slice_idx, time_idx])
    # return (image - image.min()) / (image.max() - image.min())
    # 提取并归一化图像
    image = np.abs(data[slice_idx, time_idx])
    img_norm = (image - image.min()) / (image.max() - image.min())
    brightness_factor = 3
    img_brightened = np.clip(img_norm * brightness_factor, 0, 1)
    return img_brightened

def load_config(config_file):
    """加载YAML配置文件"""
    try:
        with open(config_file, 'r') as file:
            return yaml.safe_load(file)
    except Exception as e:
        raise Exception(f"配置文件错误: {str(e)}")

def rotate_image(img):
    """顺时针旋转90度"""
    return np.rot90(img, k=-1)

def visualize_results(config_path, last_name):
    """可视化对比结果（自动适应方法数量）"""
    # 加载配置
    config = load_config(config_path)['my_config']
    
    # 加载参考数据
    ref_image = None
    if config.get('reference_file'):
        try:
            ref_image = load_4d_data(config['reference_file'], config)
        except Exception as e:
            print(f"Error loading reference: {str(e)}")

    # 加载对比数据（自动适应新增方法）
    comp_images = []
    valid_methods = []
    for fpath, title in zip(config['comparison_files'], config['comparison_titles']):
        try:
            img = load_4d_data(fpath, config)
            comp_images.append(img)
            valid_methods.append(title)
        except Exception as e:
            print(f"Error loading {title}: {str(e)}")
            comp_images.append(np.zeros((192,192)))  # 错误时填充空白

    # 统一图像尺寸
    base_shape = (192, 192)
    if ref_image is not None:
        ref_image = center_crop(ref_image, base_shape)
    comp_images = [center_crop(img, base_shape) for img in comp_images]
    
    # 计算差异图
    error_images = []
    if ref_image is not None:
        for comp_img in comp_images:
            error = np.abs(ref_image - comp_img)
            error_images.append((error - error.min()) / (error.max() - error.min()))
    
    # 动态创建布局
    num_methods = len(valid_methods)
    num_cols = num_methods + 1  # 参考图像占第一列
    
    # 创建画布（根据方法数量自动调整宽度）
    fig = plt.figure(figsize=(num_cols*3, 9))
    gs = gridspec.GridSpec(3, num_cols, height_ratios=[3, 1, 1], 
                          hspace=0.15, wspace=0.05)
    roi = config['roi_coordinates']
    
    # ========== 第一行：原始图像 ==========
    # 参考图像
    ax_ref = plt.subplot(gs[0, 0])
    if ref_image is not None:
        rotated_ref = rotate_image(ref_image)
        im_ref = ax_ref.imshow(rotated_ref, cmap='gray', 
                             vmin=config['display_range'][0], 
                             vmax=config['display_range'][1])
    ax_ref.set_title("Reference", fontsize=12, pad=10)
    ax_ref.axis('off')
    
    # 对比方法图像
    for col in range(1, num_cols):
        ax = plt.subplot(gs[0, col])
        img = comp_images[col-1]
        rotated_img = rotate_image(img)
        ax.imshow(rotated_img, cmap='gray',
                 vmin=config['display_range'][0],
                 vmax=config['display_range'][1])
        ax.set_title(valid_methods[col-1], fontsize=12, pad=10)
        ax.axis('off')
    
    # ========== 第二行：ROI区域 ==========
    # 参考ROI
    ax_ref_roi = plt.subplot(gs[1, 0])
    if ref_image is not None:
        ref_roi = ref_image[roi[1]:roi[3], roi[0]:roi[2]]
        rotated_ref_roi = rotate_image(ref_roi)
        ax_ref_roi.imshow(rotated_ref_roi, cmap='gray',
                         vmin=config['display_range'][0],
                         vmax=config['display_range'][1])
    ax_ref_roi.set_ylabel("ROI", fontsize=10, rotation=0, 
                         labelpad=20, va='center', ha='right')
    ax_ref_roi.axis('off')
    
    # 方法ROI
    for col in range(1, num_cols):
        ax = plt.subplot(gs[1, col])
        comp_roi = comp_images[col-1][roi[1]:roi[3], roi[0]:roi[2]]
        rotated_roi = rotate_image(comp_roi)
        ax.imshow(rotated_roi, cmap='gray',
                 vmin=config['display_range'][0],
                 vmax=config['display_range'][1])
        ax.axis('off')
    
    # ========== 第三行：差异图 ==========
    if ref_image is not None:
        # 参考位置留空
        plt.subplot(gs[2, 0]).axis('off')
        
        # 方法差异图
        for col in range(1, num_cols):
            ax = plt.subplot(gs[2, col])
            error_img = error_images[col-1]
            rotated_error = rotate_image(error_img)
            im_error = ax.imshow(rotated_error, cmap='viridis',
                                vmin=config['error_display_range'][0],
                                vmax=config['error_display_range'][1])
            ax.axis('off')
            if col == 1:
                ax.set_ylabel("Error", fontsize=10, rotation=0, 
                             labelpad=20, va='center', ha='right')
    
    # ========== 颜色条 ==========
    plt.subplots_adjust(right=0.85)
    if ref_image is not None:
        cax1 = fig.add_axes([0.88, 0.65, 0.015, 0.25])
        plt.colorbar(im_ref, cax=cax1, label='Intensity')
        
        cax2 = fig.add_axes([0.88, 0.15, 0.015, 0.25])
        plt.colorbar(im_error, cax=cax2, label='Error')
    
    # 保存到指定目录
    output_dir = "result_img"
    os.makedirs(output_dir, exist_ok=True)
    
    plt.savefig(
        os.path.join(output_dir, f'result_rotated_s{config["slice_index"]}_t{config["time_index"]}_{last_name}.png'),
        dpi=300,
        bbox_inches='tight'
    )
    plt.close()

In [46]:
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy import ndimage

def center_crop(image, target_shape):
    """自定义中心裁剪函数"""
    h, w = image.shape
    target_h, target_w = target_shape
    start_h = (h - target_h) // 2
    start_w = (w - target_w) // 2
    return image[start_h:start_h+target_h, start_w:start_w+target_w]

def load_4d_data(file_path, config):
    """处理四维MRI数据 (slices=118, time=18, H=192, W=192)"""
    data = np.load(file_path)
    
    if data.shape != (118, 18, 192, 192):
        raise ValueError(f"数据维度错误: 应为 (118,18,192,192)，实际得到 {data.shape}")
    
    slice_idx = np.clip(config['slice_index'], 0, 117)
    time_idx = np.clip(config['time_index'], 0, 17)
    
    image = np.abs(data[slice_idx, time_idx])
    img_norm = (image - image.min()) / (image.max() - image.min())
    brightness_factor = config.get('brightness_factor', 3)
    return np.clip(img_norm * brightness_factor, 0, 1)

def load_config(config_file):
    """加载配置文件"""
    try:
        with open(config_file, 'r') as file:
            return yaml.safe_load(file)
    except Exception as e:
        raise Exception(f"配置文件错误: {str(e)}")

def rotate_image(img):
    """顺时针旋转90度"""
    return np.rot90(img, k=-1)

def process_roi(roi_img, config):
    """ROI处理（缩放+旋转）"""
    target_size = config['roi_processing']['target_size']
    order = config['roi_processing']['interpolation_order']
    
    try:
        # 计算缩放因子
        zoom_factor = (
            target_size[0]/roi_img.shape[0], 
            target_size[1]/roi_img.shape[1]
        )
        # 插值缩放
        resized = ndimage.zoom(roi_img, zoom_factor, order=order)
        # 在process_roi函数中添加调试输出
        print(f"输入尺寸: {roi_img.shape} → 输出尺寸: {resized.shape}")
        # 旋转图像
        return rotate_image(resized)
    except Exception as e:
        print(f"ROI处理错误: {str(e)}")
        return np.zeros(target_size)

def visualize_results(config_path, last_name):
    """主可视化函数"""
    config = load_config(config_path)['my_config']
    
    # 加载数据
    ref_image, comp_images = None, []
    try:
        if config.get('reference_file'):
            ref_image = load_4d_data(config['reference_file'], config)
        
        for fpath, title in zip(config['comparison_files'], config['comparison_titles']):
            comp_images.append(load_4d_data(fpath, config))
    except Exception as e:
        print(f"数据加载错误: {str(e)}")
        return

    # 统一尺寸
    base_shape = (192, 192)
    if ref_image is not None:
        ref_image = center_crop(ref_image, base_shape)
    comp_images = [center_crop(img, base_shape) for img in comp_images]
    
    # 计算差异图
    error_images = []
    if ref_image is not None:
        for comp_img in comp_images:
            error = np.abs(ref_image - comp_img)
            error_images.append((error - error.min()) / (error.max() - error.min()))
    
    # 创建布局
    num_cols = len(comp_images) + 1
    fig = plt.figure(figsize=(num_cols*3, 9))
    gs = gridspec.GridSpec(3, num_cols, height_ratios=[3, 1, 1], hspace=0.15, wspace=0.05)
    roi = config['roi_coordinates']
    
    # ===== 第一行：原始图像 =====
    ax_ref = plt.subplot(gs[0, 0])
    if ref_image is not None:
        ax_ref.imshow(rotate_image(ref_image), cmap='gray', 
                     vmin=config['display_range'][0], 
                     vmax=config['display_range'][1])
    ax_ref.set_title("Reference", fontsize=12)
    ax_ref.axis('off')
    
    for col, (img, title) in enumerate(zip(comp_images, config['comparison_titles']), 1):
        ax = plt.subplot(gs[0, col])
        ax.imshow(rotate_image(img), cmap='gray',
                 vmin=config['display_range'][0],
                 vmax=config['display_range'][1])
        ax.set_title(title, fontsize=12)
        ax.axis('off')
    
    # ===== 第二行：ROI区域 =====
    ax_ref_roi = plt.subplot(gs[1, 0])
    if ref_image is not None:
        ref_roi = ref_image[roi[1]:roi[3], roi[0]:roi[2]]
        processed_roi = process_roi(ref_roi, config)
        ax_ref_roi.imshow(processed_roi, cmap='gray',
                         vmin=config['display_range'][0],
                         vmax=config['display_range'][1])
    ax_ref_roi.set_ylabel("ROI", fontsize=10, rotation=0, labelpad=20, va='center')
    ax_ref_roi.axis('off')
    
    for col, img in enumerate(comp_images, 1):
        ax = plt.subplot(gs[1, col])
        roi_img = img[roi[1]:roi[3], roi[0]:roi[2]]
        processed_roi = process_roi(roi_img, config)
        ax.imshow(processed_roi, cmap='gray',
                 vmin=config['display_range'][0],
                 vmax=config['display_range'][1])
        ax.axis('off')
    
    # ===== 第三行：误差图 =====
    if ref_image is not None:
        plt.subplot(gs[2, 0]).axis('off')
        
        for col, error_img in enumerate(error_images, 1):
            ax = plt.subplot(gs[2, col])
            processed_error = process_roi(error_img, config)
            ax.imshow(processed_error, cmap='viridis',
                     vmin=config['error_display_range'][0],
                     vmax=config['error_display_range'][1])
            ax.axis('off')
            if col == 1:
                ax.set_ylabel("Error", fontsize=10, rotation=0, labelpad=20, va='center')
    
    # 保存结果
    output_dir = "result_img"
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(
        os.path.join(output_dir, f'result_s{config["slice_index"]}_t{config["time_index"]}_{last_name}.png'),
        dpi=300, 
        bbox_inches='tight'
    )
    plt.close()

# if __name__ == "__main__":
#     visualize_results('config.yml', 'experiment1')

In [6]:
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy import ndimage

# 添加字体配置（关键修改）
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']  # 替换为系统中存在的字体，如 Linux 常见的 DejaVu Sans

def center_crop(image, target_shape):
    """自定义中心裁剪函数"""
    h, w = image.shape
    target_h, target_w = target_shape
    start_h = (h - target_h) // 2
    start_w = (w - target_w) // 2
    return image[start_h:start_h+target_h, start_w:start_w+target_w]

def load_4d_data(file_path, config):
    """处理四维MRI数据 (slices=118, time=18, H=192, W=192)"""
    data = np.load(file_path)
    
    if data.shape != (118, 18, 192, 192):
        raise ValueError(f"数据维度错误: 应为 (118,18,192,192)，实际得到 {data.shape}")
    
    slice_idx = np.clip(config['slice_index'], 0, 117)
    time_idx = np.clip(config['time_index'], 0, 17)
    
    image = np.abs(data[slice_idx, time_idx])
    img_norm = (image - image.min()) / (image.max() - image.min())
    brightness_factor = config.get('brightness_factor', 3)
    return np.clip(img_norm * brightness_factor, 0, 1)

def load_config(config_file):
    """加载配置文件"""
    try:
        with open(config_file, 'r') as file:
            return yaml.safe_load(file)
    except Exception as e:
        raise Exception(f"配置文件错误: {str(e)}")

def rotate_image(img):
    """顺时针旋转90度"""
    return np.rot90(img, k=-1)

def process_roi(roi_img, config):
    """ROI处理（保持尺寸一致）"""
    target_size = config['roi_processing'].get('target_size', (192, 192))  # 保持与主图一致
    order = config['roi_processing'].get('interpolation_order', 1)
    
    try:
        # 计算缩放因子
        zoom_factor = (
            target_size[0]/roi_img.shape[0], 
            target_size[1]/roi_img.shape[1]
        )
        # 插值缩放
        resized = ndimage.zoom(roi_img, zoom_factor, order=order)
        return rotate_image(resized)
    except Exception as e:
        print(f"ROI处理错误: {str(e)}")
        return np.zeros(target_size)

def visualize_results(config_path, last_name):
    """主可视化函数"""
    config = load_config(config_path)['my_config']
    
    # 加载数据
    ref_image, comp_images = None, []
    try:
        if config.get('reference_file'):
            ref_image = load_4d_data(config['reference_file'], config)
        
        for fpath, title in zip(config['comparison_files'], config['comparison_titles']):
            comp_images.append(load_4d_data(fpath, config))
    except Exception as e:
        print(f"数据加载错误: {str(e)}")
        return

    # 统一尺寸
    base_shape = (192, 192)
    if ref_image is not None:
        ref_image = center_crop(ref_image, base_shape)
    comp_images = [center_crop(img, base_shape) for img in comp_images]
    
    # 计算差异图
    error_images = []
    if ref_image is not None:
        for comp_img in comp_images:
            error = np.abs(ref_image - comp_img)
            error_images.append((error - error.min()) / (error.max() - error.min()))
    
    # 创建布局
    num_cols = len(comp_images) + 1
    fig = plt.figure(figsize=(num_cols*3, 9))
    gs = gridspec.GridSpec(3, num_cols, height_ratios=[1, 1, 1], hspace=0.1, wspace=0.05)  # 调整高度比例
    roi = config['roi_coordinates']
    
    # ===== 第一行：原始图像 =====
    ax_ref = plt.subplot(gs[0, 0])
    if ref_image is not None:
        ax_ref.imshow(rotate_image(ref_image), cmap='gray', 
                     vmin=config['display_range'][0], 
                     vmax=config['display_range'][1])
    ax_ref.set_title("Reference", fontsize=12)
    ax_ref.axis('off')
    
    for col, (img, title) in enumerate(zip(comp_images, config['comparison_titles']), 1):
        ax = plt.subplot(gs[0, col])
        ax.imshow(rotate_image(img), cmap='gray',
                 vmin=config['display_range'][0],
                 vmax=config['display_range'][1])
        ax.set_title(title, fontsize=12)
        ax.axis('off')
    
    # ===== 第二行：ROI区域 =====
    ax_ref_roi = plt.subplot(gs[1, 0])
    if ref_image is not None:
        ref_roi = ref_image[roi[1]:roi[3], roi[0]:roi[2]]
        processed_roi = process_roi(ref_roi, config)
        ax_ref_roi.imshow(processed_roi, cmap='gray',
                         vmin=config['display_range'][0],
                         vmax=config['display_range'][1])
    ax_ref_roi.set_ylabel("ROI", fontsize=10, rotation=0, labelpad=20, va='center')
    ax_ref_roi.axis('off')
    
    for col, img in enumerate(comp_images, 1):
        ax = plt.subplot(gs[1, col])
        roi_img = img[roi[1]:roi[3], roi[0]:roi[2]]
        processed_roi = process_roi(roi_img, config)
        ax.imshow(processed_roi, cmap='gray',
                 vmin=config['display_range'][0],
                 vmax=config['display_range'][1])
        ax.axis('off')
    
    # ===== 第三行：误差图 =====
    if ref_image is not None:
        plt.subplot(gs[2, 0]).axis('off')
        
        for col, error_img in enumerate(error_images, 1):
            ax = plt.subplot(gs[2, col])
            processed_error = process_roi(error_img, config)
            ax.imshow(processed_error, cmap='viridis',
                     vmin=config['error_display_range'][0],
                     vmax=config['error_display_range'][1])
            ax.axis('off')
            if col == 1:
                ax.set_ylabel("Error", fontsize=10, rotation=0, labelpad=20, va='center')
    
    # 保存结果
    output_dir = "result_img"
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(
        os.path.join(output_dir, f'result_s{config["slice_index"]}_t{config["time_index"]}_{last_name}.png'),
        dpi=300, 
        bbox_inches='tight'
    )
    plt.close()

# if __name__ == "__main__":
#     visualize_results('config.yml', 'experiment1')

In [7]:
if __name__ == "__main__":
    # visualize_results('figconfig_zzy.yml')
    # last_name = 'vista_all_10'
    # visualize_results('figconfig_zzy_vista_1.yml',last_name)
    # create_comparison_figure('figconfig_zzy_vista_1.yml',last_name)
    last_name = 'r8_0323_2'
    visualize_results('figconfig_zzy.yml',last_name)